In [1]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=False)


In [3]:
model_ort.device

device(type='cpu')

In [4]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt")

In [12]:
%%timeit 
outputs = model_ort.generate(**inputs, num_beams=3, num_return_sequences=3)

668 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
tokenizer.batch_decode(outputs)

20.3 µs ± 680 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
%%timeit
outputs = tokenizer.batch_decode(model_ort.generate(**inputs))

419 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
tokenizer.batch_decode(outputs)

19.9 µs ± 474 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# compare with pytorch

In [16]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoModel,
    AutoTokenizer
)

In [17]:
model_pt = AutoModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2")

In [18]:
model_pt.eval()
tmp = 2

In [19]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt")

In [21]:
%%timeit
outputs = model_pt.generate(**inputs, num_beams=3, num_return_sequences=3)

2.23 s ± 510 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
outputs = model_pt.generate(**inputs)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


909 ms ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
model_pt_cuda = model_pt.to('cuda')

In [28]:
inputs_cuda = inputs.to('cuda')

In [29]:
%%timeit
outputs = model_pt_cuda.generate(**inputs_cuda, num_beams=3, num_return_sequences=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


339 ms ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
outputs = model_pt_cuda.generate(**inputs_cuda)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


278 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
